**last update: 08/17/2024**  

This notebook is for transforming CMIP6 data(.nc) to .csv file. Then check the time and position range.

# Package

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr

# CMIP data process

In [1]:
# Set the file locations for historical and future data
historical_loc = "../data/CMIP5_sl/historical_all_nc/"
future_loc = "../data/CMIP5_sl/future_nc/"

# Define the model range
model_list = list(range(17, 22))

# Define the geographic region near Kenya
region = [36, 38, 4, 6]

def read_CMIP5(historical_loc, future_loc, model_id):
    historical_file = f"{historical_loc}historical_sl_{model_id}.nc"
    future_file = f"{future_loc}future_sl_{model_id}.nc"

    # Read the NetCDF files using xarray
    pre_his = xr.open_dataset(historical_file).pr
    pre_fut = xr.open_dataset(future_file).pr

    return pre_his, pre_fut

# Function to calculate spatial average
def spatial_avg(pre_his, pre_fut, basin):
    lon_min, lon_max, lat_min, lat_max = basin

    pre_his_df = pre_his.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max)).mean(dim=['lat', 'lon']).to_dataframe()
    pre_fut_df = pre_fut.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max)).mean(dim=['lat', 'lon']).to_dataframe()

    pre_df = pd.concat([pre_his_df, pre_fut_df])
    pre_df.rename(columns={'pr': 'pre_raw'}, inplace=True)

    return pre_df

# Function to convert cftime to standard datetime
def convert_to_datetimeindex(times):
    return pd.Index([pd.Timestamp(time.isoformat()) for time in times])

# Function to compute yearly precipitation
def compute_yearly(pre_his, pre_fut, basin):
    pre_df = spatial_avg(pre_his, pre_fut, basin)
    
    # Convert cftime to datetime
    times = convert_to_datetimeindex(pre_his.time.to_index().append(pre_fut.time.to_index()))
    
    # Add years and months columns
    pre_df['years'] = times.year
    pre_df['months'] = times.month
    pre_df['days_in_month'] = times.days_in_month  # Get the number of days in each month

    # Calculate monthly precipitation in mm/month
    pre_df['monthly_prep'] = pre_df['pre_raw'] * pre_df['days_in_month'] * 86400
    
    # Aggregate to annual data
    yearly_prep = pre_df.groupby('years').agg(year_prep_raw=('monthly_prep', 'sum')).reset_index()

    return yearly_prep



################################## Analysis

# for model_id in model_list:
#     pre_his, pre_fut = read_CMIP5(historical_loc, future_loc, model_id)
#     yearly_prep = compute_yearly(pre_his, pre_fut, region)
    
#     # Export data
#     export_path = f"E:/Whisky/2024summer/UGVR_2024summer/data/CMIP5_s/annual_pr_{model_id}_1850-2100.csv"
#     yearly_prep.to_csv(export_path, index=False)

# Example for a specific model (optional)
# model_id = 3
# pre_his, pre_fut = read_CMIP5(historical_loc, future_loc, model_id)
# yearly_prep = compute_yearly(pre_his, pre_fut, ken)
# export_path = f"E:/Whisky/2024summer/UGVR_2024summer/data/CMIP5_s/annual_pr_{model_id}_1850-2100.csv"
# yearly_prep.to_csv(export_path, index=False)


combined_df = pd.DataFrame()

for model_id in model_list:
    pre_his, pre_fut = read_CMIP5(historical_loc, future_loc, model_id)
    yearly_prep = compute_yearly(pre_his, pre_fut, region)
    
    # Rename the column to the corresponding model name
    yearly_prep.rename(columns={'year_prep_raw': f'model{model_id}'}, inplace=True)
    
    if combined_df.empty:
        # Initialize the combined DataFrame with the first model's data
        combined_df = yearly_prep
    else:
        # Merge the current model's data with the combined DataFrame on 'years'
        combined_df = pd.merge(combined_df, yearly_prep, on='years', how='outer')


In [2]:
combined_df = pd.DataFrame()

for model_id in model_list:
    pre_his, pre_fut = read_CMIP5(historical_loc, future_loc, model_id)
    yearly_prep = compute_yearly(pre_his, pre_fut, region)
    
    # Rename the column to the corresponding model name
    yearly_prep.rename(columns={'year_prep_raw': f'model{model_id}'}, inplace=True)
    
    if combined_df.empty:
        # Initialize the combined DataFrame with the first model's data
        combined_df = yearly_prep
    else:
        # Merge the current model's data with the combined DataFrame on 'years'
        combined_df = pd.merge(combined_df, yearly_prep, on='years', how='outer')

# # Export the combined data to a CSV file
# export_path = "E:/Whisky/2024summer/UGVR_2024summer/data/CMIP5_sl/combined_annual_pr_1850-2100.csv"
# combined_df.to_csv(export_path, index=False)

In [2]:
export_path = "E:/Whisky/2024summer/UGVR_2024summer/data/CMIP5_sl/test_sl_annual_pr_1850-2099.csv"
combined_df.to_csv(export_path, index=False)

In [3]:
combined_df

,years,model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,model11,model12,model13,model14,model15,model16
0,2000,472.455378,240.183497,1056.041494,442.108002,222.156317,570.225894,192.895486,238.930884,625.311731,391.718614,408.031697,1503.848354,1082.611859,942.130986,1724.425014,278.690284
1,2001,265.618346,574.285126,1308.282330,532.550158,194.192360,1066.804708,187.578552,341.084010,1290.686520,427.283396,828.456002,1497.090827,1281.620677,1038.841532,1750.863375,636.857110
2,2002,369.318766,316.351813,1535.178824,413.949811,209.841345,1407.625234,170.178596,375.284331,1074.353067,617.863803,711.258016,1263.474577,1067.794129,697.367217,1850.912970,414.562935
3,2003,360.208038,299.957020,928.300709,690.188621,331.801181,1077.595128,212.666412,179.872793,614.497556,655.401168,416.288460,1457.708891,1179.160148,1028.134273,1675.343744,548.058739
4,2004,427.388148,500.729709,1167.854674,578.727639,306.140566,1120.753560,198.409989,177.581730,776.731098,615.639559,480.940988,1589.839102,1098.525280,1236.514979,1505.869179,493.593873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2095,543.364498,362.637865,1623.728635,532.406163,894.515169,1883.031606,311.837107,498.180391,815.644610,761.546345,286.210800,1404.932409,1102.484756,806.699823,1703.710224,643.527906
96,2096,607.111503,446.403455,1307.813233,662.311847,393.770876,1588.515309,131.491352,201.855175,1054.512192,605.264190,397.504443,1675.121312,1283.122986,948.282853,1598.095605,472.835363
97,2097,680.073583,314.676203,1510.944698,735.405690,579.183146,1267.101446,270.646493,671.032383,1344.443459,444.383753,591.566852,1682.679538,1557.270972,1242.561651,1504.055587,777.316122
98,2098,476.145275,731.760640,1490.615830,446.410536,859.954721,1515.305621,233.822840,175.059947,944.562462,544.305081,523.249442,1689.109973,1617.361460,1154.419378,1085.540754,848.650749


# check

In [4]:
# Function to get the time range of a NetCDF file
def get_time_range(nc_file):
    ds = xr.open_dataset(nc_file)
    time_var = ds.time  # Assuming the time variable is named 'time'
    time_start = str(time_var[0].values)
    time_end = str(time_var[-1].values)
    return time_start, time_end

# Function to check time range for all models
def check_all_time_ranges(historical_loc, future_loc, model_list):
    for model_id in model_list:
        historical_file = f"{historical_loc}historical_sl_{model_id}.nc"
        future_file = f"{future_loc}future_sl_{model_id}.nc"

        his_time_start, his_time_end = get_time_range(historical_file)
        fut_time_start, fut_time_end = get_time_range(future_file)

        print(f"Model {model_id}:")
        print(f"  Historical: {his_time_start} to {his_time_end}")
        print(f"  Future: {fut_time_start} to {fut_time_end}\n")

# Run the function to check time ranges for all models
check_all_time_ranges(historical_loc, future_loc, model_list)


Model 1:
  Historical: 2000-01-16T12:00:00.000000000 to 2014-12-16T12:00:00.000000000
  Future: 2015-01-16T12:00:00.000000000 to 2099-12-16T12:00:00.000000000

Model 2:
  Historical: 2000-01-16T12:00:00.000000000 to 2014-12-16T12:00:00.000000000
  Future: 2015-01-16T12:00:00.000000000 to 2099-12-16T12:00:00.000000000

Model 3:
  Historical: 2000-01-16 12:00:00 to 2014-12-16 12:00:00
  Future: 2015-01-16 12:00:00 to 2099-12-16 12:00:00

Model 4:
  Historical: 2000-01-16 12:00:00 to 2014-12-16 12:00:00
  Future: 2015-01-16 12:00:00 to 2099-12-16 12:00:00

Model 5:
  Historical: 2000-01-16 12:00:00 to 2014-12-16 12:00:00
  Future: 2015-01-16 12:00:00 to 2099-12-16 12:00:00

Model 6:
  Historical: 2000-01-16 12:00:00 to 2014-12-16 12:00:00
  Future: 2015-01-16 12:00:00 to 2099-12-16 12:00:00

Model 7:
  Historical: 2000-01-16T12:00:00.000000000 to 2014-12-16T12:00:00.000000000
  Future: 2015-01-16T12:00:00.000000000 to 2099-12-16T12:00:00.000000000

Model 8:
  Historical: 2000-01-16 12:00:

In [5]:
# Function to get the latitude and longitude range of a NetCDF file
def get_lat_lon_range(nc_file):
    ds = xr.open_dataset(nc_file)
    lat = ds.lat  # Assuming the latitude variable is named 'lat'
    lon = ds.lon  # Assuming the longitude variable is named 'lon'
    lat_min = float(lat.min().values)
    lat_max = float(lat.max().values)
    lon_min = float(lon.min().values)
    lon_max = float(lon.max().values)
    return lat_min, lat_max, lon_min, lon_max

# Function to check lat/lon range for all models
def check_all_lat_lon_ranges(historical_loc, future_loc, model_list):
    for model_id in model_list:
        historical_file = f"{historical_loc}historical_sl_{model_id}.nc"
        future_file = f"{future_loc}future_sl_{model_id}.nc"

        his_lat_min, his_lat_max, his_lon_min, his_lon_max = get_lat_lon_range(historical_file)
        fut_lat_min, fut_lat_max, fut_lon_min, fut_lon_max = get_lat_lon_range(future_file)

        print(f"Model {model_id}:")
        print(f"  Historical Lat: {his_lat_min} to {his_lat_max}, Lon: {his_lon_min} to {his_lon_max}")
        print(f"  Future Lat: {fut_lat_min} to {fut_lat_max}, Lon: {fut_lon_min} to {fut_lon_max}\n")

# Run the function to check lat/lon ranges for all models
check_all_lat_lon_ranges(historical_loc, future_loc, model_list)


Model 1:
  Historical Lat: 4.375 to 5.625, Lon: 36.5625 to 36.5625
  Future Lat: 4.375 to 5.625, Lon: 36.5625 to 36.5625

Model 2:
  Historical Lat: 4.207777988548681 to 5.14283974829786, Lon: 36.5625 to 37.5
  Future Lat: 4.207777988548681 to 5.14283974829786, Lon: 36.5625 to 37.5

Model 3:
  Historical Lat: 5.04670442015714 to 5.04670442015714, Lon: 36.0 to 37.125
  Future Lat: 5.04670442015714 to 5.04670442015714, Lon: 36.0 to 37.125

Model 4:
  Historical Lat: 5.04670442015714 to 5.04670442015714, Lon: 36.0 to 37.125
  Future Lat: 5.04670442015714 to 5.04670442015714, Lon: 36.0 to 37.125

Model 5:
  Historical Lat: 4.185920533189158 to 4.185920533189158, Lon: 36.5625 to 36.5625
  Future Lat: 4.185920533189158 to 4.185920533189158, Lon: 36.5625 to 36.5625

Model 6:
  Historical Lat: 4.240837696335078 to 5.1832460732984345, Lon: 36.25 to 37.5
  Future Lat: 4.240837696335078 to 5.1832460732984345, Lon: 36.25 to 37.5

Model 7:
  Historical Lat: 4.244101319570208 to 5.742019423919015, L